<a href="https://colab.research.google.com/github/yyoonyoon/201935291-/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트
앙상블 학습 중 가장 유명하고 안정적인 성능을 제공
여러개의 결정트리를 랜덤하게 만들고 각 결정트리의

1. 훈련하기 위한 데이터를 랜덤하게 만듦.
입력한 훈련에이터에서 랜덤하게 샘플 추출.
중복허용.
이렇게 만들어진 샘플을 부트스트랩 샘플 이라 함.
기본적으로 훈려넷트의 크기와 동일하게 설정함.

2. 부트스트랩 샘플로 결정트릴를 훈련
각 노드를 분할 할 때 전체 특성중에서 일부 특성을 무작위로 고른다음 이 중에서 최선의 분할을 찾음.

3. 정해진 수 만큼 위 과정을 반복.


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

램덤포레스트와 비슷하게 동작
부트스트랩 샘플을 사용하지 않고 각 결정 트리를 만들 때 전체훈련 세트를 사용
노드를 분할 할 때 가장 좋은 분할을 찾지 않고 랜덤으로 분할함.

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻음
이 예제는 특성이 많지 않아 두 모델이 차이가 크지 않음.
보통 엑스트라 트리가 무작위성이 좀 더 큼.

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식.
사이킷런에서는 기본적으로 깊이가 3인 결정트리 100개 사용
깊이가 얕은 걀정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

그레이디언트란 이름이 붙은 이유는 경사 하강법을 사용하여 트리를 앙상블에 추가하기 때문
분류에서는 로지스틱 손실함수를 사용
회귀에서는 평균제곱 오차 함수를 사용
경사 하강법이 손실함수를 산으로 정의하고 가장낮은 곳을 찾아 내려온ㄴ 과정이라면 그레이디언트 부스팅은 결정트릴를 계속 추가하면서 가장 낮은 곳을 찾아 이동함.
경사 하강법에서 손실함수의 낮은 ~~~

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대적합에 매우강함. estimators 는 100이 기본값.

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

모댈을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모댈을 만들고 최종적으로 생성된 모든 모델을 하나로 합침.
정형 데이터를 다루는 머신러닝 알고리즘 중에서 가장 인기가 높은 알고리즘
입력 특성을 256구간으로 나누므로 노드를 분할 할 ㄸ ㅐ 최적의 분할을 매우 빠르게 찾음
256개의 구간중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함.
즉 어떤 샘플에 누라ㄱ된 값이 있담면 떼어노은 구간의 값을 대신 사용함.
따라서 입력에 누락된 특성이 있어도 따로 전처리 노필요

히스토그램 데이터를 일정한 구간으로 쪼개서 막대그래프로 표현.
입력 데이터의 각 특성을 256개의 구간으로 분할
계산을 256번만 하면 되므로 효율적이고 빠름.

In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [14]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM
ms에서 만듦


In [15]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
